# Patent text translation

## Introduction

As shown in the notebook `text_data_exploration.ipynb`, EP full text data is available for patents in English, French and German.

In [16]:
!pip3.6 install nltk
!pip3.6 install googletrans

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 55 kB 743 kB/s  eta 0:00:01
     |████████████████████████████████| 42 kB 296 kB/s  eta 0:00:01
     |████████████████████████████████| 938 kB 26.2 MB/s eta 0:00:01
     |████████████████████████████████| 53 kB 672 kB/s  eta 0:00:01
     |████████████████████████████████| 65 kB 703 kB/s  eta 0:00:01
     |████████████████████████████████| 98 kB 1.7 MB/s  eta 0:00:01
  Created wheel for googletrans: filename=googletrans-3.0.0-py3-none-any.whl size=15737 sha256=e32bf04533120798d1fa016915ee903fd205e82747e3446acf643b3f8a625fc2
  Stored in directory: /home/antoine/.cache/pip/wheels/06/a1/90/75375ea9aff253a4803b2356007f64aecb6ce2f1e46a0aa9ba
  Created wheel for contextvars: filename=contextvars-2.4-py3-non

In [3]:
import pandas as pd
from nltk import sent_tokenize
from googletrans import Translator

In [18]:
# according to the doc, step required
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/antoine/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

## Load data

In [4]:
data_sample = pd.read_csv('../data/ep_full_text_database/2020_edition/EP0000000.txt',
                          sep = '\t',  header = None)

In [5]:
data_sample.columns = ['publication_authority', # will always have the value "EP"
                'publication_number', # a seven-digit number
                'publication_kind', # see https://www.epo.org/searching-for-patents/helpful-resources/first-time-here/definitions.html for help.
                'publication_date', # in format YYYY-MM-DD
                'language_text_component', # de, en, fr; xx means unknown
                'text_type', # TITLE, ABSTR, DESCR, CLAIM, AMEND, ACSTM, SREPT, PDFEP
                'text' # it contains, where appropriate, XML tags for better structure. You will find the DTD applicable to all parts of the publication at: http://docs.epoline.org/ebd/doc/ep-patent-document-v1-5.dtd
               ]
data_sample.head()

,publication_authority,publication_number,publication_kind,publication_date,language_text_component,text_type,text
0,EP,2,A1,1978-12-20,de,TITLE,"Tetrahydrofuran-Derivate, Verfahren zu ihrer H..."
1,EP,2,A1,1978-12-20,en,TITLE,"Tetrahydrofurane derivatives, processes for th..."
2,EP,2,A1,1978-12-20,fr,TITLE,"Dérivés du tétrahydrofuranne, leurs procédés d..."
3,EP,2,A1,1978-12-20,de,ABSTR,"<p id=""pa01"" num=""0001"">Die vorliegende Erfind..."
4,EP,2,A1,1978-12-20,de,DESCR,"<p id=""p0001"" num=""0001"">Die vorliegende Erfin..."


## Translating patent text

In [6]:
data_sample.language_text_component.value_counts()

en    546740
de    444896
fr    336723
xx     51157
Name: language_text_component, dtype: int64

In [30]:
# Get a sample patent title in French
condition1 = data_sample.text_type == 'TITLE'
condition2 = data_sample.language_text_component == 'fr'
df = data_sample[condition1 & condition2]['text'].to_frame()
example_fr_xml = df.iloc[111]['text']
example_fr_xml

'Dispositif pour soulever'

In [31]:
def translate(text_to_translate, source_language, target_language):
    """Machine translation using the Google translate API"""
    translator = Translator()
    translatedText = translator.translate(text_to_translate, src=source_language, dest=target_language)
    return translatedText.text

In [32]:
translate(example_fr_xml, 'fr', 'en')

'Device for lifting'